# 将原始数据集自然语言的形式转化为逻辑表达式

In [ ]:
from typing import List
from typing import Optional
from typing import Dict

import os
import json
from pathlib import Path

import httpx
from openai import OpenAI

import requests
from tqdm import tqdm
import re

## 单条prompt测试

In [ ]:
# 文件路径
file_path = "prompts/DataFountain/translation.txt"

# 请求的 URL
url = "https://api.moonshot.cn/v1/files"

headers = {
    'Authorization': 'sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw'
}

# 准备文件数据
files = {'file': open(file_path, 'rb')}

# 发起 POST 请求
response = requests.post(url, headers=headers, files=files)

# 打印响应内容
print(response.status_code)
print(response.text)

In [ ]:
client = OpenAI(
    api_key="sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw",
    base_url="https://api.moonshot.cn/v1",
)

file_object = client.files.create(file=Path("prompts/DataFountain/translation.txt"), purpose="file-extract")
file_content = client.files.content(file_id=file_object.id).text

In [ ]:
print(file_content)

In [16]:
datas = []
with open('data/法律/DISC-Law-SFT-Triplet-released.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        data = json.loads(line.strip())
        datas.append(data)

with open('data/法律/DISC-json.json', 'w', encoding='utf-8') as f:
    json.dump(datas, f, indent=4, ensure_ascii=False)

In [ ]:
print(datas[0]["reference"])

In [ ]:
answers = []

In [ ]:
import time


# def call_api_with_rate_limit_handling(client, messages, model="moonshot-v1-32k", max_retries=5):
#     retries = 0
#     while retries < max_retries:
#         try:
#             completion = client.chat.completions.create(
#                 model=model,
#                 messages=messages,
#                 temperature=0,
#             )
#             return completion
#         except requests.exceptions.HTTPError as e:
#             print(e.response.status_code)
#             if e.response.status_code == 429:
#                 print('429')
#                 error_data = e.response.json()
#                 if error_data.get('error', {}).get('type') == 'rate_limit_reached_error':
#                     print(f"Rate limit reached. Retrying in {error_data.get('error', {}).get('message', '1')} seconds...")
#                     time.sleep(int(error_data.get('error', {}).get('message').split(' ')[-2]) + 1)
#                     retries += 1
#                 else:
#                     raise  # Re-raise the exception if it's not a rate limit error
#             else:
#                 raise  # Re-raise the exception if it's not a 429 error
#     raise Exception("Max retries reached. API call failed.")

def call_api_with_rate_limit_handling(client, messages, model="moonshot-v1-32k", max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            completion = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0,
            )
            return completion
        except Exception as e:
            if '429' in str(e):  # 检查异常消息中是否包含429
                print("Rate limit reached. Retrying in 1 second...")
                time.sleep(1)  # 等待1秒钟
                retries += 1
            else:
                raise  # Re-raise the exception if it's not a 429 error
    raise Exception("Max retries reached. API call failed.")

In [ ]:
# 替换内容
replacement = "根据《刑法》第一百一十四条的规定，被告人白某某以危险方法危害公共安全，尚未造成严重后果。根据案件事实，被告人白某某在公交车上与司机发生争执，并用手拉拽档杆和司机的胳膊，导致公交车失控撞向停放的轿车和路灯杆，最终造成公交车、轿车、路灯杆以及福锅记饭店牌匾的损坏。根据鉴定结果，本案损失共计人民币19,342.6元。\n综上所述，根据《刑法》第一百一十四条的规定，被告人白某某的行为构成以危险方法危害公共安全罪。根据该罪的刑罚幅度，可判处三年以上十年以下有期徒刑。\n需要注意的是，以上仅为根据提供的案件事实和相关法律条文进行的推理，最终的判决结果还需根据法庭审理的具体情况和法官的判断做出。"

# 使用正则表达式替换 [[RULES]]
content = re.sub(r'\[\[RULES\]\]', replacement, file_content)

messages = [
    {
        "role": "system",
        "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"
    },
    {
        "role": "system",
        "content": content,
    },
    {
        "role": "user",
        "content": "你好，请根据txt内容生成回答。"
    }
]

completion = call_api_with_rate_limit_handling(client, messages, "moonshot-v1-32k", 5)

print(completion.choices[0].message.content)

In [ ]:
for dicts in tqdm(data[84], desc='rule转化逻辑表达式进度'):
    # 替换内容
    replacement = dicts['rule_text']

    # 使用正则表达式替换 [[RULES]]
    content = re.sub(r'\[\[RULES\]\]', replacement, file_content)

    messages = [
        {
            "role": "system",
            "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。请将内容输出为可解析的json格式。"
        },
        {
            "role": "system",
            "content": content,
        },
        {
            "role": "user",
            "content": "你好，请根据txt内容生成回答。"
        }
    ]

    completion = call_api_with_rate_limit_handling(client, messages, "moonshot-v1-32k", 5)

    print(dicts['rule_id'])
    answers.append(completion.choices[0].message.content)

In [ ]:
with open('data/DF/answers_v3.json', 'w', encoding='utf-8') as f:
    json.dump(answers, f, ensure_ascii=False, indent=4)

In [ ]:
print(answers[0])

## cache方法传输测试

In [ ]:
client = OpenAI(
    api_key="sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw",
    base_url="https://api.moonshot.cn/v1",
)

In [ ]:
def upload_files(files: List[str], cache_tag: Optional[str] = None) -> List[Dict[str, Any]]:
    """
    upload_files 会将传入的文件（路径）全部通过文件上传接口 '/v1/files' 上传，并获取上传后的
    文件内容生成文件 messages。每个文件会是一个独立的 message，这些 message 的 role 均为
    system，Kimi 大模型会正确识别这些 system messages 中的文件内容。

    如果你设置了 cache_tag 参数，那么 upload_files 还会将你上传的文件内容存入 Context Cache
    上下文缓存中，后续你就可以使用这个 Cache 来对文件内容进行提问。当你指定了 cache_tag 的值时，
    upload_files 会生成一个 role 为 cache 的 message，通过这个 message，你可以引用已被缓存
    的文件内容，这样就不必每次调用 `/v1/chat/completions` 接口时都要把文件内容再传输一遍。

    注意，如果你设置了 cache_tag 的值，你需要把 upload_files 返回的 messages 放置在请求
    `/v1/chat/completions` 接口时 messages 参数列表的第一位（实际上，我们推荐不管是否启用
    cache_tag，都将 upload_files 返回的 messages 放置在 messages 列表的头部）。

    关于 Context Caching 的具体信息，可以访问这里：

    https://platform.moonshot.cn/docs/api/caching

    :param files: 一个包含要上传文件的路径的列表，路径可以是绝对路径也可以是相对路径，请使用字符串
        的形式传递文件路径。
    :param cache_tag: 设置 Context Caching 的 tag 值，你可以将 tag 理解为自定义的 Cache 名称，
        当你设置了 cache_tag 的值，就意味着启用 Context Caching 功能，默认缓存时间是 300 秒，每次
        携带缓存进行 `/v1/chat/completions` 请求都将刷新缓存存活时间（300 秒）。
    :return: 一个包含了文件内容或文件缓存的 messages 列表，请将这些 messages 加入到 Context 中，
        即请求 `/v1/chat/completions` 接口时的 messages 参数中。
    """
    messages = []

    # 对每个文件路径，我们都会上传文件并抽取文件内容，最后生成一个 role 为 system 的 message，并加入
    # 到最终返回的 messages 列表中。
    for file in files:
        file_object = client.files.create(file=Path(file), purpose="file-extract")
        file_content = client.files.content(file_id=file_object.id).text
        messages.append({
            "role": "system",
            "content": file_content,
        })

    if cache_tag:
        # 当启用缓存（即 cache_tag 有值时），我们通过 HTTP 接口创建缓存，缓存的内容则是前文中通过文件上传
        # 和抽取接口生成的 messages 内容，我们为这些缓存设置一个默认的有效期 300 秒（通过 ttl 字段），并
        # 为这个缓存打上标记，标记值为 cache_tag（通过 tags 字段）。
        r = httpx.post(f"{client.base_url}caching",
                       headers={
                           "Authorization": f"Bearer {client.api_key}",
                       },
                       json={
                           "model": "moonshot-v1",
                           "messages": messages,
                           "ttl": 300,
                           "tags": [cache_tag],
                       })

        if r.status_code != 200:
            raise Exception(r.text)

        # 创建缓存成功后，我们不再需要将文件抽取后的内容原封不动地加入 messages 中，取而代之的是，我们可以设置一个
        # role 为 cache 的消息来引用我们已缓存的文件内容，只需要在 content 中指定我们给 Cache 设定的 tag 即可，
        # 这样可以有效减少网络传输的开销，即使是多个文件内容，也只需要添加一条 message，保持 messages 列表的清爽感。
        return [{
            "role": "cache",
            "content": f"tag={cache_tag};reset_ttl=300",
        }]
    else:
        return messages

# 带缓存的转换

In [12]:
from typing import List
from typing import Optional
from typing import Dict

import os
import json
from pathlib import Path

import httpx
from openai import OpenAI

import requests

import time
import re
from tqdm import tqdm

### 建立缓存

In [13]:
# 你的API密钥
api_key = "sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw"

# 读取文本文件内容
with open('prompts/DataFountain/translation.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()

# 创建缓存的请求数据
data = {
    "model": "moonshot-v1",
    "messages": [
        {
            "role": "system",
            "content": file_content
        }
    ],
    "name": "example_cache",
    "ttl": 3600  # 缓存有效期，单位为秒
}

# 发送POST请求创建缓存
response = requests.post(
    url="https://api.moonshot.cn/v1/caching",
    headers={
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    },
    json=data
)

# 打印响应内容
print(json.loads(response.text))

{'id': 'cache-eysgt1nyospi11ania31', 'object': 'cache_object', 'model': 'moonshot-v1', 'messages': [{'role': 'system', 'content': '任务描述：你将被给予一个规则的描述。你的任务是将规则解析为一阶逻辑公式，并输出为可解析的 JSON 格式。\n一阶逻辑公式的语法定义如下：\n1） 确定规则中涉及的所有谓词。\n2） 定义每个谓词的含义和参数。\n3） 根据规则的逻辑结构，使用一阶逻辑的连接词（如∧、∨、⊕、¬、→、↔等）和量词（如∀、∃等）构建逻辑表达式。\n------\n下面是一个例子：\n规则：\n安全生产协调司的职责：根据安全监管总局领导指示和有关规定，组织协调安全监察专员赶赴事故现场参与事故应急救援和事故调查处理工作。\n###\n{\n    "rule": "安全生产协调司的职责：根据安全监管总局领导指示和有关规定，组织协调安全监察专员赶赴事故现场参与事故应急救援和事故调查处理工作。",\n    "predicates":\n    [\n        "指示：Instruction(Leader, Regulation) ::: 表示领导发出的指示和相关规定。",\n        "参与：Participate(Inspector, Scene, Task) ::: 表示安全监察专员在事故现场参与某项任务。",\n        "赶赴：GoTo(Inspector, Scene) ::: 表示安全监察专员赶赴事故现场。",\n        "事故：Accident(Scene) ::: 表示发生在某个场景的事故。"\n    ],\n    "logicexpression":\n    [\n        "∀s ∀i ∀r (Accident(s) ∧ Instruction(Leader, r) → (GoTo(i, s) ∧ Participate(i, s, EmergencyRescue) ∧ Participate(i, s, Investigation)))"\n    ]\n}\n------\n下面是一个例子：\n规则：\n交通运输应急保障：安全监管总局建立全国主要危险化学品从业单位的交通地理

In [11]:
# 你的API密钥
api_key = "sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw"

# 缓存ID，假设从创建缓存的响应中获取
cache_id = "cache-ey5p3ifxfuei11gkxpii"

# 发送DELETE请求删除缓存
response = requests.delete(
    url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
    headers={
        "Authorization": f"Bearer {api_key}"
    }
)

# 打印响应状态码
print(response.status_code)

# 如果需要，也可以打印响应内容
print(response.text)

404
{"error":{"message":"context cache not found: cache-ey5p3ifxfuei11gkxpii","type":"resource_not_found_error"}}


In [14]:
# 你的API密钥
api_key = "sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw"


# 读取文本文件内容
def load_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()


# 创建缓存
def create_cache(file_content):
    data = {
        "model": "moonshot-v1",
        "messages": [
            {
                "role": "system",
                "content": file_content
            }
        ],
        "name": "example_cache",
        "ttl": 3600  # 缓存有效期，单位为秒
    }
    response = requests.post(
        url="https://api.moonshot.cn/v1/caching",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    cache_response = json.loads(response.text)
    return cache_response['id']

In [15]:
# 检查缓存是否存在且未过期
def check_cache(cache_id):
    response = requests.get(
        url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
        headers={
            "Authorization": f"Bearer {api_key}"
        }
    )
    if response.status_code == 200:
        return True
    return False


# 重新加载数据并更新缓存
def reload_and_update_cache(file_path, cache_id):
    new_file_content = load_file_content(file_path)
    data = {
        "model": "moonshot-v1",
        "messages": [
            {
                "role": "system",
                "content": new_file_content
            }
        ],
        "name": "example_cache",
        "ttl": 3600  # 缓存有效期，单位为秒
    }
    response = requests.put(
        url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    return response.status_code == 200


# 使用缓存内容并添加问题
def use_cache_with_question(cache_id, question):
    data = {
        "model": "moonshot-v1-32k",
        "messages": [
            {
                "role": "cache",
                "content": f"cache_id={cache_id};reset_ttl=3600",
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    response = requests.post(
        url="https://api.moonshot.cn/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    return response.json()['choices'][0]['message']['content']

### 主流程

In [18]:
# 主流程
file_path = 'prompts/DataFountain/translation.txt'
file_content = load_file_content(file_path)
print(f"File{file_path} success read")
cache_id = create_cache(file_content)
print(f"cache id:{cache_id}")
print("cache cuccess")

# # 模拟缓存过期
# time.sleep(3600)

# 检查缓存是否存在且未过期
if not check_cache(cache_id):
    print("检查不到缓存")
    # 重新加载数据并更新缓存
    if reload_and_update_cache(file_path, cache_id):
        print("缓存更新成功")
    else:
        print("缓存更新失败")

print("缓存已存在")

with open('data/法律/DISC-json.json', 'r', encoding='utf-8') as file:
    answer = json.load(file)

judgement = []
for idx, value in tqdm(enumerate(answer[:1000]), total=len(answer), desc="Processing"):
    context = answer[idx]["reference"]

    # 使用缓存内容并添加问题
    question = f"请回答关于文件内容的问题，其中[[CONTEXT]]代表的数据为{context}"
    response = use_cache_with_question(cache_id, question)
    judgement.append({
        "id": idx,
        "judgement": response
    })

with open('data/法律/law_nl2logic_v2.json', 'w', encoding='utf-8') as f:
    json.dump(judgement, f, indent=4, ensure_ascii=False)

Fileprompts/DataFountain/translation.txt success read
cache id:cache-eyt4ni3r71ui11caaezi
cache cuccess
缓存已存在


Processing:   1%|          | 178/16000 [40:39<60:14:41, 13.71s/it]


ProxyError: HTTPSConnectionPool(host='api.moonshot.cn', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by ProxyError('Unable to connect to proxy', RemoteDisconnected('Remote end closed connection without response')))